# Bonus Task

We need to develop a machine learning model trained on the processsed dataset to detect which passengers were transported by the anomaly. We will predict on provided test.csv.

We have already cleaned_dataset.csv which is a cleaned dataset that we can use as the training dataset. 

We also need to clean test.csv, which we will clean in the same way we cleaned train taking values from train ONLY. (so we only clean test and don't like influence our outcomes based on 'knowing' the test dataset)

We will do this by creating a 'cleaning function', then we will reclean our original unclean dataset using that function making sure that we get a similar result as our cleaned_dataset, and then apply that cleaning function on test.csv

In [1]:
# import libraries here

import pandas as pd
import numpy as np


In [ ]:
#Import our required datasets

cleaned = pd.read_csv('../Task2/cleaned_dataset.csv')
train = pd.read_csv('../Data/train.csv')
test = pd.read_csv('../Data/test.csv')

In [ ]:
#Create our cleaning function (I will be looking in #AstroGoodBoy_notebook_cleaning for reference and getting the same code in)
np.random.seed(69)

def clean(data):
    cleanData = data.copy()

    #PassengerId + Name column stuff
    cleanData['GroupId'] = data['PassengerId'].str[:4]
    cleanData['GroupSize'] = data.groupby(data['PassengerId'].str[:4]).transform('size')

    #HomePlanet and Destination stuff
    cleanData['HomePlanet'] = cleanData.groupby(cleanData['GroupId'])['HomePlanet'].transform('first')
    cleanData['HomePlanet'] = cleanData['HomePlanet'].fillna('Earth')
    cleanData['Destination'] = cleanData['Destination'].fillna('TRAPPIST-1e')

    #Cabin stuff
    cleanData[['Cabin P1', 'Cabin P2', 'Cabin P3']] = cleanData['Cabin'].str.split("/", expand = True)
    cleanData['Cabin P2'] = cleanData['Cabin P2'].astype('Int64')
    cleanData['Cabin P1'] = cleanData['Cabin P1'].replace('T', 'F')
    cleanData['Cabin P1'] = cleanData['Cabin P1'].fillna('F')

    #We needed a randomfill function to do a bunch of our work
    #We need to somehow make sure we get random values from train.csv and put in test.csv (we not allowed to access test.csv values)
    #Instead to solve this issue, we will change our randomFill code to always take values from our already CLEANED dataset!
    def randomFill(column):
      SGMRW = column.copy()
      indices = SGMRW[SGMRW.isna()].index
      randomVals = np.random.choice(cleaned[column.name], size = SGMRW.isna().sum(), replace = True)
      SGMRW.loc[indices] = randomVals
      return SGMRW

    #More cabin stuff
    cleanData['Cabin P2'] = cleanData.groupby(cleanData['Cabin P1'])['Cabin P2'].transform(lambda x: randomFill(x))
    cleanData['Cabin P3'] = cleanData.groupby(cleanData['GroupId'])['Cabin P3'].transform('first')
    cleanData['Cabin P3'] = randomFill(cleanData['Cabin P3'])
    cleanData['Cabin'] = cleanData['Cabin P1'].astype(str) + '/' + cleanData['Cabin P2'].astype(str) + '/' + cleanData['Cabin P3'].astype(str)

    #Cryosleep, VIP and some spendings stuff
    cleanData['VIP'] = (cleanData['VIP']).astype(bool)
    cleanData['VIP'] = cleanData['VIP'].fillna(False)
    #lmao this part looking very inefficient now but whatever 
    tempData = cleanData.copy()
    tempData['RoomService'] = tempData['RoomService'].fillna(0.0)
    tempData['FoodCourt'] = tempData['FoodCourt'].fillna(0.0)
    tempData['ShoppingMall'] = tempData['ShoppingMall'].fillna(0.0)
    tempData['Spa'] = tempData['Spa'].fillna(0.0)
    tempData['VRDeck'] = tempData['VRDeck'].fillna(0.0)
    indices = cleanData[(cleanData['CryoSleep'].isna() == True) & ((cleanData['RoomService'] + cleanData['FoodCourt'] + cleanData['ShoppingMall'] + cleanData['Spa'] + cleanData['VRDeck']) != 0) & ((tempData['RoomService'] + tempData['FoodCourt'] + tempData['ShoppingMall'] + tempData['Spa'] + tempData['VRDeck']) == 0)].index
    cleanData.loc[indices, 'CryoSleep'] = True
    cleanData.loc[indices, 'RoomService':'VRDeck'] = 0.0
    indices = cleanData[(cleanData['CryoSleep'].isna() == True) & ((cleanData['RoomService'] + cleanData['FoodCourt'] + cleanData['ShoppingMall'] + cleanData['Spa'] + cleanData['VRDeck']) == 0)].index
    cleanData.loc[indices, 'CryoSleep'] = True
    cleanData['CryoSleep'] = (cleanData['CryoSleep']).astype(bool)
    cleanData['CryoSleep'] = cleanData['CryoSleep'].fillna(False)
    cleanData.loc[cleanData['CryoSleep'] == True, 'RoomService':'VRDeck'] = 0

    #Age and spendings stuff
    spendings = cleanData[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].fillna(0).sum(axis = 1)
    indices = cleanData[((spendings == 0) & (cleanData['CryoSleep'] == False) & (cleanData['Age'].isna() == True))].index
    randomVals = np.random.choice([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], size = len(cleanData[((spendings == 0) & (cleanData['CryoSleep'] == False) & (cleanData['Age'].isna() == True))]), replace = True)
    cleanData.loc[indices, 'Age'] = randomVals
    cleanData['Age'] = randomFill(cleanData['Age'])
    cleanData['RoomService'] = randomFill(cleanData['RoomService'])
    cleanData['FoodCourt'] = randomFill(cleanData['FoodCourt'])
    cleanData['ShoppingMall'] = randomFill(cleanData['ShoppingMall'])
    cleanData['Spa'] = randomFill(cleanData['Spa'])
    cleanData['VRDeck'] = randomFill(cleanData['VRDeck'])
    cleanData.loc[cleanData['Age'] < 13, 'RoomService':'VRDeck'] = 0

    #Final cleaning steps
    cleanData.drop(columns = ['Name', 'Cabin'] , inplace = True)
    cleanData['TotalSpendings'] = cleanData['RoomService'] + cleanData['FoodCourt'] + cleanData['ShoppingMall'] + cleanData['Spa'] + cleanData['VRDeck']
    
    return cleanData

Crazy that so much of our time spent on the other notebook and so much of our code there can be compressed to just this